<a href="https://colab.research.google.com/github/arpan0009e/Arpan-AICTE-AI-Travel-Planner/blob/main/AI_TRAVEL_PLANNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Title: AI Travel Planner for Students

Problem Statement:

Planning trips is time-consuming and expensive for students with limited budgets.

Existing travel apps often provide generic, non-personalized suggestions.

Objective:

Develop an AI-driven app to generate personalized, budget-friendly, and efficient travel itineraries.

Scope:

Uses Gemini AI to create custom, day-by-day travel plans.

Integrates map and location data to optimize routes and logistics.

Tailored to student profiles based on destination, budget, and personal interests.

System Architecture:

Frontend: Streamlit (Python)

Backend: Google Gemini API & Google Maps API

Workflow:

User inputs trip details (destination, budget, interests).

AI generates a personalized daily itinerary.

App displays the plan with an interactive map of all locations.
'''


In [1]:
# Run this cell to install all dependencies
%pip install -q streamlit pyngrok google-generativeai pandas requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.4 MB/s eta 0:00:00


In [5]:
%%writefile app.py

import streamlit as st
import google.generativeai as genai
import pandas as pd
import requests
import json
import os

# --- Page Configuration ---
st.set_page_config(
    page_title="AI Travel Planner",
    page_icon="✈️",
    layout="wide"
)

# --- Function Definitions ---

def fetch_places(destination, interest, api_key):
    """Fetches real places from Google Places API."""
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={interest} in {destination}&key={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        results = response.json().get('results', [])
        places = []
        for place in results[:5]:  # Get top 5 results
            places.append({
                "name": place.get('name'),
                "rating": place.get('rating', 'N/A'),
                "location": place.get('geometry', {}).get('location', {})
            })
        return places
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching places: {e}")
        return []

def generate_itinerary(destination, duration, budget, interests, places_data, api_key):
    """Calls the Gemini API to generate an itinerary."""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('models/gemini-2.5-pro')
    generation_config = genai.GenerationConfig(response_mime_type="application/json")

    prompt = f"""
    You are an expert travel planner. Create a detailed {duration}-day travel itinerary for {destination} with a budget of ${budget}, focusing on interests in {', '.join(interests)}.
    Use ONLY the locations provided in this list: {json.dumps(places_data)}. Do not invent new places.
    Return a valid JSON object with a single key "itinerary", which is a list of days. Each day must have keys: "day", "title", and "activities".
    Each activity must have keys: "name", "description", and "location" (with "lat" and "lng"). Ensure the location data is copied accurately from the provided list.
    """
    try:
        response = model.generate_content(prompt, generation_config=generation_config)
        return json.loads(response.text)
    except Exception as e:
        st.error(f"AI itinerary generation failed. Error: {e}")
        return None

# --- Main App Interface ---

st.title("Your AI Travel Planner ✈️")

# Use secrets for API keys
try:
    GEMINI_KEY = st.secrets["GEMINI_API_KEY"]
    MAPS_KEY = st.secrets["MAPS_API_KEY"]
except FileNotFoundError:
    st.error("Secrets file not found. Please ensure your API keys are set up correctly.")
    st.stop()

# --- Sidebar for User Inputs ---
with st.sidebar:
    st.header("Your Trip Details")
    destination = st.text_input("Destination", "Kyoto, Japan")
    duration_days = st.number_input("Number of Days", min_value=1, max_value=14, value=3)
    budget = st.text_input("Budget ($)", "1200")
    interests = st.multiselect(
        "Select Your Interests",
        ['Temples and Shrines', 'Gardens', 'Traditional Food', 'Anime and Manga', 'Hiking'],
        default=['Gardens', 'Traditional Food']
    )
    generate_button = st.button("Generate Itinerary ✨")

# --- Main Logic and Display ---
if generate_button:
    if not destination or not interests:
        st.warning("Please enter a destination and at least one interest.")
    else:
        with st.spinner("Finding the best spots..."):
            all_places = []
            for interest in interests:
                all_places.extend(fetch_places(destination, interest, MAPS_KEY))

        if not all_places:
            st.error("Could not find any locations for the selected interests. Please try again.")
        else:
            with st.spinner("AI is crafting your personalized itinerary... 🧠"):
                itinerary_data = generate_itinerary(destination, duration_days, budget, interests, all_places, GEMINI_KEY)
                st.session_state.itinerary = itinerary_data

# --- Display Results ---
if 'itinerary' in st.session_state and st.session_state.itinerary:
    st.success("Your itinerary is ready!")
    itinerary_data = st.session_state.itinerary

    col1, col2 = st.columns([1.5, 1])

    with col1:
        for day in itinerary_data.get('itinerary', []):
            st.subheader(f"📍 Day {day['day']}: {day['title']}")
            for activity in day.get('activities', []):
                with st.expander(f"**{activity['name']}**"):
                    st.markdown(activity['description'])
                    maps_link = f"https://www.google.com/maps/search/?api=1&query={activity['location']['lat']},{activity['location']['lng']}"
                    st.link_button("View on Google Maps ↗️", maps_link)

    with col2:
        st.subheader("🗺️ Your Trip Map")
        locations = []
        for day in itinerary_data.get('itinerary', []):
            for activity in day.get('activities', []):
                loc = activity.get('location')
                if loc and 'lat' in loc and 'lng' in loc:
                    locations.append({"name": activity['name'], "lat": loc['lat'], "lng": loc['lng']})

        if locations:
            map_df = pd.DataFrame(locations)
            st.map(map_df, latitude='lat', longitude='lng', size=20, zoom=11)
else:
    st.info("Enter your trip details in the sidebar to get started!")

Overwriting app.py


In [7]:
import os
from google.colab import userdata
from pyngrok import ngrok

# Create the .streamlit/secrets.toml file to pass API keys to the app
os.makedirs(".streamlit", exist_ok=True)
with open(".streamlit/secrets.toml", "w") as f:
    f.write(f'GEMINI_API_KEY = "{userdata.get("GEMINI_API_KEY")}"\n')
    f.write(f'MAPS_API_KEY = "{userdata.get("MAPS_API_KEY")}"\n')

# Terminate any existing ngrok tunnels
ngrok.kill()

# Setup and run ngrok
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(8501)
    print("\n🎉 Your Streamlit app is live!")
    print(f"Click here: {public_url}\n")
except Exception as e:
    print(f"❌ Error launching ngrok. Please ensure your NGROK_AUTH_TOKEN is correct in Colab secrets. Error: {e}")

# Run the Streamlit app
!streamlit run app.py --server.port 8501


🎉 Your Streamlit app is live!
Click here: NgrokTunnel: "https://chirographic-belen-unanaemic.ngrok-free.dev" -> "http://localhost:8501"






  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.21.6.29:8501

  Stopping...
  Stopping...
